In [4]:

import textract

text = textract.process("./文档3.docx")
# with open('./a.txt') as f:
#     text=f.read()

In [5]:
import re
import pandas as pd
def BasicInformation(text,preprocessed=False):
    result={}
    if preprocessed:
        text2=text
    else:
        text0=re.sub('\n','=',re.sub('\xe2\x80\xa8','=',text))# remove /n and \xe2\x80\xa8 stand for /n in chinese
        text01=re.sub('\xef\xbc\x9a',':',text0)
        text02=re.sub('\xef\xbc\xa3\xef\xbc\xb4','CT',text01)
        text1=re.sub('\xc2\xa0','',text02)
        text03=re.sub('=窗体底端=','',text1)
        text04=re.sub('=窗体顶端=','',text03)
        text2=re.sub('=INCLUDEPICTURE.*?MERGEFORMATINET =','',text04)
#     result['MRN']=re.search('病案号.*?=+(\d+)=+',text2).group(1)
    result['DOB']=re.search('出生日期:=+(\d+-\d+-\d+)=+',text2).group(1)
    
    result['AdmitTime']=re.search('入院日期.*?=+(.*?)=+',text2).group(1)
    result['ChiefComplaint']=re.search('主 诉:(.*?)=+现病史',text2).group(1)
    result['PrimaryDiagnose']=' '.join(re.findall('=+(.*?)=+ICD\d+=+.*?=+\d\d\d\d-\d\d-\d\d=',re.search('入院诊断(.*?)补充诊断',text2).group(1)))
    result['ORTime']=re.search('手术日期:(.*?)=+手术医生',text2).group(1)
#     result['ORDoctor']=re.search('手术医生:(.*?)=+麻醉方案',text2).group(1)
    result['ORNote']=re.search('手术经过:=+(.*?)=+\d+-\d+-\d+ \d+:\d+',text2).group(1)
    temp=re.search('(见病变位于.*?大小约(.*?)cm.*?)\xe3\x80\x82',result['ORNote'])# \xe3\x80\x82 equal to .
    result['LesionLocation']=temp.group(1)
    result['LesionSize']=temp.group(2)
    try:
        result['BloodLoss']=re.search('出血约(\d+ml)',result['ORNote']).group(1)
    except:
        result['BloodLoss']='Unknown'
    result['FinialDiagnose']=re.search('出院诊断.*?=(.*?)=出院医嘱',text2).group(1)
    return [result,text2]

In [6]:
pd.DataFrame({1:BasicInformation(text)[0]})

,1
AdmitTime,2013-05-21 09:46
BloodLoss,300ml
ChiefComplaint,垂体腺瘤开颅术后10年余，视力下降1年，全身乏力1月。
DOB,1967-04-01
FinialDiagnose,鞍区占位:复发垂体腺瘤 甲状腺功能低下症 肾上腺皮质功能不全 继发性糖尿病
LesionLocation,见病变位于鞍上、鞍后，并长入三脑室，实性，质地软，起源于垂体，垂体柄受压位于肿瘤后方，大小约...
LesionSize,3x3.5x3.8
ORNote,全麻插管后，患者取仰卧位，头稍前屈，左偏5度，头架固定头部，常规消毒铺单。取原过中线右额颞小...
ORTime,2013-06-03
PrimaryDiagnose,鞍区占位:复发垂体腺瘤？ 甲状腺功能低下症 肾上腺皮质功能不全 继发性糖尿病


# report

In [15]:
def ResultDf(text,label,preprocessed=False,test='BloodTest'):
    '''
    label:游离甲状腺激素 (FT4) or  白蛋白 (should the exactly the same as in the note)
    test={'BloodTest','RadiologyReport'}
    if test='RadiologyReport'
        label: {CT,磁共振,黑白B超,心脏特殊检查,放射...}
    '''
    result={}
    if preprocessed:
        text2=text
    else:
        text0=re.sub('\n','=',re.sub('\xe2\x80\xa8','=',text))# remove /n and \xe2\x80\xa8 stand for /n in chinese
        text01=re.sub('\xef\xbc\x9a',':',text0)
        text02=re.sub('\xef\xbc\xa3\xef\xbc\xb4','CT',text01)
        text1=re.sub('\xc2\xa0','',text02)
        text03=re.sub('=窗体底端=','',text1)
        text04=re.sub('=窗体顶端=','',text03)
        text2=re.sub('=INCLUDEPICTURE.*?MERGEFORMATINET =','',text04)
    result={}
    n=0
    #MRN=re.search('病案号.*?=+(\d+)=+',text2).group(1)
    MRN=None
    if test=='BloodTest':
        for TestTime,Result,Unit,Range in re.findall('报告日期:=+(\d\d\d\d-\d\d-\d\d).*?'+
                                                    re.sub('[(]','\(',re.sub('[)]','\)',label))+
                                                    '=+([-+><]?\d*\.*\d+)=+[a-z]=+(.*?)=+(.*?)=',text2):
            result[n]={'MRN':MRN,'Label':label,'TestTime':TestTime,'Result':Result,'Unit':Unit,'Range':Range}
            n+=1
    elif test=='RadiologyReport':        
        AllReport=re.search('=+[a-z]=+.*?=+.*?=+(\d+-\d+-\d+ \d+:\d+ ?=+.*?=+检查项目:.*?)手术资料 =+',text2).group(1)
        for TestTime,_,Label,Result,Impression in re.findall('(\d+-\d+-\d+ \d+:\d+) ?=+'+
                                      label+
                                      '=+(.?){0,39}=+完成 ?=+检查项目:=+(.*?)=+.*?检查所见:=+(.*?)=+检查结论/诊断:=+(.*?)=+',AllReport):
            result[n]={'MRN':MRN,'Label':Label,'TestTime':TestTime,'FullReport':Result,'Impression':Impression}
            n+=1
    else:
        print "test={'BloodTest','RadiologyReport'}"
    return pd.DataFrame(result)

In [16]:
ResultDf(text,label='游离甲状腺激素 (FT4)')

,0,1,2,3
Label,游离甲状腺激素 (FT4),游离甲状腺激素 (FT4),游离甲状腺激素 (FT4),游离甲状腺激素 (FT4)
MRN,None,None,None,None
Range,12-22,12-22,12-22,12-22
Result,20.000,20.450,13.520,12.690
TestTime,2013-06-04,2013-06-05,2013-05-22,2013-05-22
Unit,pmol/L,pmol/L,pmol/L,pmol/L


In [17]:
ResultDf(text,label='CT',test='RadiologyReport')

,0,1,2
FullReport,鞍底下陷，蝶鞍扩大，蝶窦呈全鞍型，未见蝶上筛房；筛窦粘膜增厚，鼻道、鼻甲未见明显异常。鞍区见...,原“鞍区占位:复发垂体腺瘤？”患者术后复查，无老片对比，现片示:右额部呈术后改变，局部骨质...,原“鞍区占位:复发垂体腺瘤？”患者术后复查，与PACS2013-06-03老片对比，现片示:...
Impression,1 鞍区占位病变，脑积水，脑白质水肿；2 脑CTA未见明显异常。,"""鞍区占位:复发垂体腺瘤？""术后改变。必要时复查。",“鞍区占位:复发垂体腺瘤？”术后改变，与PACS2013-06-03老片对比，右额颅板下积血...
Label,"头部CT:CT颅脑血管成像CTA,CT鞍区HRCT三维成像",头部CT:CT颅脑平扫(螺旋),头部CT:CT颅脑平扫(螺旋)
MRN,None,None,None
TestTime,2013-05-23 11:51,2013-06-02 10:58,2013-06-05 14:49
